In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re
from string import punctuation
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [2]:
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")
snowball = SnowballStemmer(language="russian")
mystem = Mystem()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zer0nu11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data_path = 'C:\\Users\\zer0nu11\\Documents\\grad\\skolkovohack2022\\data\\for_hack_2022\\'
test_path = 'C:\\Users\\zer0nu11\\Documents\\grad\\skolkovohack2022\\data\\for_hack_2022\\test\\'

In [4]:
jobs_labels = ['JobId','Status','Name','Region','Description']
test_jobs_labels = ['JobId','Status','Name','Region','Description','nan1','nan2','nan3']
candidates_labels = ['CandidateId', 'Position', 'Sex', 'Citizenship', 'Age', 'Salary',
       'Langs', 'DriverLicense', 'Subway', 'Skills', 'Employment', 'Schedule',
       'CandidateRegion','DateCreated','JobId','CandidateStatusId','Status']
candidates_education_labels = ['CandidateId', 'University', 'Faculty', 'GraduateYear']
candidates_workplaces_labels = ['CandidateId', 'Position', 'FromYear', 'FromMonth', 'ToYear',
       'ToMonth']

data_jobs = pd.read_csv(data_path+'data_jobs.csv',sep=';',names=jobs_labels)
data_candidates_workplaces = pd.read_csv(data_path+'data_candidates_work_places.csv',sep=';',names=candidates_workplaces_labels)
data_candidates_education = pd.read_csv(data_path+'data_candidates_education.csv',sep=';',names=candidates_education_labels)
data_candidates = pd.read_csv(data_path+'data_candidates.csv',sep=';',names=candidates_labels)

test_jobs = pd.read_csv(test_path+'test_jobs.csv',sep=';',names=test_jobs_labels)
test_candidates_workplaces = pd.read_csv(test_path+'test_candidates_workplaces.csv',sep=';')
test_candidates_education = pd.read_csv(test_path+'test_candidates_education.csv',sep=';')
test_candidates = pd.read_csv(test_path+'test_candidates.csv',sep=';')

In [5]:
# missing data
data_jobs = data_jobs.fillna('').drop(['Status'],axis=1)
data_candidates_workplaces = data_candidates_workplaces.fillna('').drop(['FromYear','FromMonth','ToYear','ToMonth'],axis=1)
data_candidates_education = data_candidates_education.fillna('').drop(['GraduateYear','University'],axis=1)
data_candidates = data_candidates.fillna('')

test_jobs = test_jobs.fillna('').drop(['Status','nan1','nan2','nan3'],axis=1)
test_candidates_workplaces = test_candidates_workplaces.fillna('').drop(['FromYear','FromMonth','ToYear','ToMonth'],axis=1)
test_candidates_education = test_candidates_education.fillna('').drop(['GraduateYear','University'],axis=1)
test_candidates = test_candidates.fillna('')

In [6]:
assert (data_jobs.columns == test_jobs.columns).all()
# assert (data_candidates.columns == test_candidates.columns).all() # data.shape > test.shape because of relation to job_id
print('Train/Test columns difference:\n',list(data_candidates.columns[~data_candidates.columns.isin(test_candidates.columns)]))
assert (data_candidates_education.columns == test_candidates_education.columns).all()
assert (data_candidates_workplaces.columns == test_candidates_workplaces.columns).all()

Train/Test columns difference:
 ['DateCreated', 'JobId', 'CandidateStatusId', 'Status']


In [7]:
def preprocess_signs(text):
    text = text.lower()
    text = re.sub(r'<.*?>', " ", text)
    text = re.sub(r'[\_+\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]', " ", text)
    text = re.sub(r'[ ]{2,}',' ',text)
    text = text.strip()
    # tokens = mystem.lemmatize(text)
    # tokens = [snowball.stem(token) for token in tokens if token not in russian_stopwords\
    #             and token != " " \
    #             and token.strip() not in punctuation ]
    # text = " ".join(tokens)
    return text

In [8]:
data_jobs[['Name','Region','Description']] = data_jobs[['Name','Region','Description']].applymap(preprocess_signs)
data_candidates_workplaces[['Position']] = data_candidates_workplaces[['Position']].applymap(preprocess_signs)
data_candidates_education[['Faculty']] = data_candidates_education[['Faculty']].applymap(preprocess_signs)
data_candidates[['Position','Citizenship','Langs','DriverLicense', \
        'Subway','Skills','CandidateRegion','Status']] = \
    data_candidates[['Position','Citizenship','Langs','DriverLicense', \
        'Subway','Skills','CandidateRegion','Status']].applymap(preprocess_signs)

test_jobs[['Name','Region','Description']] = test_jobs[['Name','Region','Description']].applymap(preprocess_signs)
test_candidates_workplaces[['Position']] = test_candidates_workplaces[['Position']].applymap(preprocess_signs)
test_candidates_education[['Faculty']] = test_candidates_education[['Faculty']].applymap(preprocess_signs)
test_candidates[['Position','Citizenship','Langs','DriverLicense', \
        'Subway','Skills','CandidateRegion']] = \
    test_candidates[['Position','Citizenship','Langs','DriverLicense', \
        'Subway','Skills','CandidateRegion']].applymap(preprocess_signs)

In [23]:
# list(test_jobs.Description.unique())
list(data_jobs.Description.unique())
# data_candidates_workplaces
# test_candidates_workplaces
# data_candidates_education
# test_candidates_education

# data_candidates
# test_candidates

['',
 'требования к опыту водительские права категории b d обязанности утренняя развозка персонала две ходки в 09 00 и в 11 00 от м пр т ветеранов до завода росэл в п шушары вечерняя развозка персонала две ходки в 17 00 и в 19 00 от завода росэл до м пр т ветеранов в течении дня выполнение поручений руководства развозка документов материалов небольшого оборудования соблюдение чистоты салона при необходимости осуществление хим чистки салона контроль за прохождением то талонов страховых полюсов комментарий',
 'требования к опыту от 5 лет обязанности развозка документов выполнение личных поручений нег менеджера гостиницы комментарий',
 'требования к опыту о р от 3 лет права водительские кат д обязанности развозка пассажиров на автобусе комментарий',
 'водитель для партнеров',
 'график день ночь 5 2 2 2 6 1 3 1 любой зп в среднем при 5 2 55тр 60 тр в мес',
 'обязанности доставка груза заказчику приемка экспедирование сдача груза грузополучателю оформление погрузочно разгрузочных документов

In [11]:
test_candidates

,CandidateId,Position,Sex,Citizenship,Age,Salary,Langs,DriverLicense,Subway,Skills,Employment,Schedule,CandidateRegion
0,8458568,специалист отдела парковочного комплекса,2,россия,50,45000,английский базовые знания||русский родной||,,,ведение документации управление производством ...,Part,,санкт петербург
1,8480519,оператор,2,россия,50,50000,русский родной||немецкий базовые знания||,,ленинский проспект,водительское удостоверение категории bcd || пр...,Part,Flex,санкт петербург
2,8511395,ученик автослесаря автомеханика,2,россия,24,50000,русский родной||английский базовые знания||,,,работоспособность || навыки коммуникации || ра...,Part,,санкт петербург
3,8511459,агент по обработке авиаперевозок,2,россия,33,60000,русский родной||,,,способность заниматься несколькими проектами |...,Part,Flex,москва
4,8530237,техник механик,2,россия,50,45000,русский родной||английский свободно владею||,,,активная позиция || пк excel word || тракторны...,Look-out,,санкт петербург
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,339,менеджер по продажам ведущий менеджер,1,россия,0,45000,русский родной||английский базовые знания||,,м петроградская,знание техники продаж умение вести переговоры ...,Full,Full,санкт петербург
196,585,менеджер по продажам закупкам,2,россия,0,30000,английский базовый||,b,проспект nbsp ветеранов,,Full,Full,санкт петербург
197,616,менеджер по продажам,2,россия,30,40000,русский базовые знания||английский могу проход...,,,,Full,Full,санкт петербург
198,622,менеджер по продажам,2,россия,30,40000,русский базовые знания||английский могу проход...,,м новочеркасская,,Full,Full,санкт петербург
